In [1]:
from BasisFilter import BasisFilter
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import os
import copy


In [3]:
BF = BasisFilter()
basis = BF.load_basis()

C:\Users\howsr\workspace\1_DataPreprocessing\basis\code\makeAllBasisCode\BasisFilter.py:36: DtypeWarning: Columns (2,68,69,70,71,98,99,100,101,128,129,130,131,158,159,160,161,188,189,190,191,218,219,220,221,248,249,250,251,278,279,280,281,308,309,310,311,338,339,340,341,368,369,370,371,398,399,400,401,428,429,430,431,458,459,460,461,488,489,490,491,518,519,520,521,548,549,550,551,578,579,580,581,608,609,610,611,638,639,640,641,668,669,670,671,698,699,700,701,728,729,730,731,758,759,760,761,788,789,790,791) have mixed types. Specify dtype option on import or set low_memory=False.
  self.basis = pd.read_csv(f'{self.path}\\MergeRegionData\\basis_for_filter.csv', encoding='utf-8')


In [37]:
ProgressNote_list = [col for col in basis.columns.tolist() if 'ProgressNote' in col]
ProgressNote_list

['ProgressNote_1',
 'ProgressNote_2',
 'ProgressNote_3',
 'ProgressNote_4',
 'ProgressNote_5',
 'ProgressNote_6',
 'ProgressNote_7',
 'ProgressNote_8',
 'ProgressNote_9',
 'ProgressNote_10',
 'ProgressNote_11',
 'ProgressNote_12',
 'ProgressNote_13',
 'ProgressNote_14',
 'ProgressNote_15',
 'ProgressNote_16',
 'ProgressNote_17',
 'ProgressNote_18',
 'ProgressNote_19',
 'ProgressNote_20',
 'ProgressNote_21',
 'ProgressNote_22',
 'ProgressNote_23',
 'ProgressNote_24',
 'ProgressNote_25',
 'ProgressNote_26',
 'ProgressNote_27']

In [69]:
bulmun_dict = {}
for col in ProgressNote_list[1:]:
    print(col)
    for i in basis.index:
        text = basis.loc[i,col]
        if i % 10000 ==0:
            print(f'{i}/130000')
        if isinstance(text,float):
            continue
        sentances = extract_sentences_with_keyword(text,'불면')
        for sentance in sentances:
            if '안내' not in sentance:
                if '주의' not in sentance:
                    if '경과' not in sentance:
                        if '교감' not in sentance:
                            if '증상' in sentance:
                                if '불편' in sentance:                                          
                                    if i in list(bulmun_dict.keys()):
                                        print(sentance)
                                        bulmun_dict[i] = bulmun_dict[i] + [col]
                                    else:
                                        bulmun_dict[i] = [col]

ProgressNote_2
0/130000
* 불면 및 미식거림등 불편증상 있으시고, 현재 감량추이도 양호하시어 동일농도로 진행 도와드림. 
*본원 다이어트 약 복용 시 불편증상: 불면증, 초기 두근거림 
양방,한방/  마지막 / 1달전 불편증상 없음,빈속 복용시 한약냄새 올라옴, 입마름, 불면 살짝
*본원 다이어트 약 복용 시 불편증상: 불면
* 불면외에 큰 불편증상 없으시며, 식욕은 초반에 비해서는 올라와서 상향 원하심. 
>> 불편증상 불면증 , 변비 기재해주셨습니다. 
*본원 다이어트 약 복용 시 불편증상: 불면
*본원 다이어트 약 복용 시 불편증상:두통, 불면증 (가끔 수면제 복용. 3~4일에 한번씩. 현재는 남은약 없고, 앞으로 처방 안받으려고 한다하심)
*본원 다이어트 약 복용 시 불편증상: 가슴 두근거림 , 불면
*본원 다이어트 약 복용 시 불편증상: 두근거림, 불면 잠시 (3~4일)
□ 불편증상 : 불면,속쓰림,땀, 열감,기타(입냄새)
*본원 다이어트 약 복용 시 불편증상:불면증
□ 불편증상 발현 시점 : 평소 불면증이 있었는데 더 심해서 힘드네요~
                                                 효과x 불편증상으로 불면 있었음
- 불편증상 : 가슴 답답한 증상 , 불면
*본원 다이어트 약 복용 시 불편증상:입마름,불면증
□ 불편증상 발현 시점 : 속쓰림과 두근거림은 3알 먹음 나타나고 불면증은 1알 먹어두 나옹
*복용 당시 불편증상: 손떨림 , 불면
감비정D4 복용 1~2회 복용, 불면 외에 여타 불편증상 없음.
*본원 다이어트 약 복용 시 불편증상: 불면증 약간 
10000/130000
 - 불편증상 여부 : 불면
*본원 다이어트 약 복용 시 불편증상: 불면증 , 입마름
초반에 불면 매우 심했음 - 약간 더 순하게 시작 권장 / 불편증상 우려 - 저녁약 안드시면 안되냐고 문의주심 
- 불편증상 여부 : 두통, 불면증
* 저녁에 불면 매우 심하심 고려. 입마름 등 추가적 불편증상도 있으셨음. 
 - 불편

In [14]:
def extract_sentences_with_keyword(text, keyword):
    sentences = text.split('\n')
    keyword_sentences = [sentence for sentence in sentences if keyword in sentence]
    
    return keyword_sentences



In [71]:
bulmun_dict

12286

In [67]:
basis['insomnia'] = basis.index.isin(list(bulmun_dict.keys())).astype(int)

In [73]:
before_inbody_dict ={}
for idx in bulmun_dict.keys():
    num = bulmun_dict[idx][0].split('_')[-1]
    if basis.loc[idx,'Weigt_'+num].isna():
        before_inbody_dict[idx] = num

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
